In [140]:
import pandas as pd
import pickle

In [141]:
# Import full parameters dataframe
params_df = pd.read_csv('/home/s212597/BOcalibration/parameters_original.csv')
params_df['include'] = False

In [142]:
# For each submodel, import and filter the 25 most changed parameters
# Change the file name accordingly for each submodel
with open('/home/s212597/BOcalibration/saved_outputs/param_space_dpx.pkl', 'rb') as f:
    param_space = pickle.load(f)

# Calculate initial sample set's inadecuacy
initial = param_space["definition"].set_index("parameter")["value"].transpose().to_frame().T
initial.reset_index(drop=True, inplace=True)
initial['inadequacy']= 20.75489

# Top performer
optimal_sample = param_space["space"].nsmallest(1, 'inadequacy')
# Getting the row with the lowest inadequacy value from optimal_samples5
second_lowest_inadequacy = optimal_sample['inadequacy'].nsmallest(1).iloc[-1]  # get the 2nd smallest value
second_lowest_inadequacy_row = optimal_sample[optimal_sample['inadequacy'] == second_lowest_inadequacy]

# Calculate differences
differences_second = (second_lowest_inadequacy_row.iloc[0] - initial.iloc[0]).drop('inadequacy')

# Sorting the differences and selecting top 25
top25 = differences_second.abs().sort_values(ascending=False).head(25)
top25_list = top25.index.tolist()
# Extract unique model names from the list
model_names = {string.split('_')[0] for string in top25_list}
# Remove characters before "beta" in each string
top25_list = [s[s.index('beta'):] if 'beta' in s else s for s in top25_list]

In [143]:
# Keep only top 25 changed parameter for each model
# Loop through DataFrame cells
for index, row in params_df.iterrows():
    cell_text = row['param']
    # Check if any string from string_list is contained in the cell_text
    for string2 in top25_list:
        if string2 in cell_text:
            if row['model'] in model_names:
                params_df.at[index, 'include'] = True

In [144]:
# Export
#params_df.to_csv("/home/s212597/BOcalibration/parameters_final25.csv", index = False)